## Getting vcfs ids

We want to get ids, that we will use in the future (after merge vcfs and phenotype table).

First - get VCFs ids.

In [1]:
import pandas as pd
import numpy as np
import re
from collections import Counter

WAY_TO = "../vcf/gvcfs/"

DATA_FOLDER = "./data"
VCF_ID_FILE = f"{DATA_FOLDER}/gvcfs.list"
PHENO_FILE = f"{DATA_FOLDER}/pheno.csv"
OLD_MAPPING_FILE = "./data/MAPPING.csv"
FOR_TRANSFORMATION_TO_SAVE = "./data/special_mapping_mappings_final_all_launches.csv"
FOR_GWAS_TO_SAVE = "./data/special_mapping_idsAndInfo_final_all_launches.csv"
MAPPING_DONT_KNOW_ABOUT_THEM = "./data/special_mapping_dont_know.txt"

In [2]:
def transform_vcf_list(string):
    try:
        string = (
            string.lower()
            .replace(WAY_TO, "")
            .replace("out.", "")
            .replace("g.vcf.gz\n", "")
            .replace("recal.", "")
            .replace("dedup.", "")
            .replace("ex_cov_", "ex-cov-")
        )
        launch_reg = re.compile("ces_([0-9])*.")
        launch_id = launch_reg.match(string).group()[len("ces_") : -1]
        id_index = string[len(launch_reg.match(string).group() + "sample_") : -1]

        id_index = id_index.split("-")[-1].split(".")[0]
        id_index = (
            id_index.replace("seq00", "")
            .replace("seq0", "")
            .replace("seq", "")
            .replace("cov", "")
            .replace("ec", "")
            .replace("m", "")
            .replace("ex", "")
        )
        print(string, launch_id, int(id_index))
        new_name = f"ces_{launch_id}.sample_{launch_id + id_index}.g.vcf.gz"
        return str(int(launch_id + id_index)), new_name
    except ValueError:
        print("KEKKK   ", end=" ")
        print(string, end=" ")
        return None


## They aren't using for now
## DEPRECATED
def bash_rename(old_name, new_name, new_id):
    return f"""
        echo -e 'Processing file {old_name}\\n\\n'
        mv {old_name} {new_name}
        gzip -d {new_name}
        /media/EXOMEDATA/software/gatk-4.1.4.0/gatk RenameSampleInVcf \\
        -I {new_name[:-3]}                     \\
        -O n{new_name[:-3]}                      \\
        --NEW_SAMPLE_NAME {new_id}
        rm -r {new_name[:-3]} 
        gzip n{new_name[:-3]}
        echo -e '\\nNew file: n{new_name[:-3]}\\n\\n\\n\\n'
    """


def bash_delete(old_name):
    return f"""
        echo -e 'Deleting {old_name}\\n\\n'
        rm -r {old_name}
        echo -e '\\nDeleted: old_name\\n\\n\\n\\n'
    """

In [3]:
with open(VCF_ID_FILE) as input_list:
    vcf_ids_files = input_list.readlines()

errors = 0
vcfs_ids = []
id_to_file = dict()
tuple_to_rename = []
for file in vcf_ids_files:
    final_index, new_file_name = transform_vcf_list(file)
    file = file.replace("\n", "")
    if final_index:
        id_to_file.update({final_index: file})
        vcfs_ids.append(final_index)
        tuple_to_rename.append((file, new_file_name, final_index))
    else:
        errors += 1
print(f"ERRORS = {errors}")

ces_104.sample_l1-1-555. 104 555
ces_104.sample_l1-10-6472. 104 6472
ces_104.sample_l1-11-6474. 104 6474
ces_104.sample_l1-12-6475. 104 6475
ces_104.sample_l1-13-6476. 104 6476
ces_104.sample_l1-14-6477. 104 6477
ces_104.sample_l1-15-6478. 104 6478
ces_104.sample_l1-16-6479. 104 6479
ces_104.sample_l1-17-6482. 104 6482
ces_104.sample_l1-18-6492. 104 6492
ces_104.sample_l1-19-6494. 104 6494
ces_104.sample_l1-2-1371. 104 1371
ces_104.sample_l1-20-6498. 104 6498
ces_104.sample_l1-21-6501. 104 6501
ces_104.sample_l1-22-6508. 104 6508
ces_104.sample_l1-23-6509. 104 6509
ces_104.sample_l1-24-6516. 104 6516
ces_104.sample_l1-3-1674. 104 1674
ces_104.sample_l1-4-1862. 104 1862
ces_104.sample_l1-5-1933. 104 1933
ces_104.sample_l1-6-1944. 104 1944
ces_104.sample_l1-7-4919. 104 4919
ces_104.sample_l1-8-6143. 104 6143
ces_104.sample_l1-9-6183. 104 6183
ces_104.sample_l2-1-6529. 104 6529
ces_104.sample_l2-10-6523. 104 6523
ces_104.sample_l2-11-6525. 104 6525
ces_104.sample_l2-12-6527. 104 6527
ces_

## ids from pheno table

Get ids from table of phenotypes

In [4]:
data = pd.read_csv(PHENO_FILE)
data.head()

,"ШТОРМ 1-да, 0-нет",от куда пациент,N ПАЦИЕНТА(КРОВЬ),"Пол ( 1-м, 2-ж)",КАРТА ПАЦ-ТА,Количество суток с начала заболевания до госпитализации,Дата начала лечения,День заболевания,"Исходы (1 умер, 2 выписка)","Степень тяжести заболевания (1-легкая, 2-средняя, 3-тяжелая и кр.тяжелая)",...,Плоскоклеточная метаплазия,Десквамация эпителия бронхов,Спазм мелких бронхов,Десквамированный альвеолярный эпителий в просветах альвеол,Амилоидные тельца,Фиброз межальвеолярных перегородок,Интерстициальный отёк,Утолщение межальвеолярных перегородок за счет склероза,Лимфо-макрофагальная инфильтрация межальвеолярных перегородок,Аспирационная пневмония (инородные частицы в просвете терминальной бронхиолы)
0,0,заря,11,1,82590.0,NaN,NaN,NaN,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,заря,309,1,89212.0,NaN,NaN,NaN,2,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,заря,373,1,104435.0,NaN,NaN,NaN,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,заря,254,1,107415.0,NaN,NaN,NaN,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,заря,73,1,108533.0,NaN,NaN,NaN,2,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
def transform(string):
    try:
        string = (
            string.lower()
            .replace(" ", "")
            .replace("(биобанк)", "")
            .replace("seq", "")
            .replace("ex_cov_", "")
        )
        return int(string)
    except ValueError:
        print(string)
        return string


ids = list(data["N ПАЦИЕНТА(КРОВЬ)"])
data["blood_id"] = data["N ПАЦИЕНТА(КРОВЬ)"].apply(transform)
print("lengthes (list and set):")
print(len(ids), len(set(ids)))

1200/1234
1221/1244
8()/67
lengthes (list and set):
853 853


In [6]:
# This is to understand below how to carry sequencers and numbers with IDs
def special_id(string):
    try:
        string = (
            string.lower().replace(" ", "").replace("(биобанк)", "").replace("seq", "")
        )
        if "ex_cov_" in string:
            return "ex_cov"
        else:
            return "l"
    except ValueError:
        return None


data["may_launch_special_id"] = data["N ПАЦИЕНТА(КРОВЬ)"].apply(special_id)

In [7]:
id_cols = [
    "НАЗВАНИЕ БИБЛИОТЕКИ (ЗАПУСКА)",
    "СЕКВЕНАТОР",
    "ДАТА ЗАПУСКА",
    "ИНДЕКС",
    "ГРУППА",
]

In [8]:
seq = data["СЕКВЕНАТОР"]  # sequence machine
date = data["ДАТА ЗАПУСКА"]  # date of sequence machine launch
lib = data["НАЗВАНИЕ БИБЛИОТЕКИ (ЗАПУСКА)"]  # name of the lib
may = data["may_launch_special_id"]
launch_series = (
    seq.str.cat(date, sep=" || ", na_rep="-")
    .str.cat(lib, sep=" || ", na_rep="-")
    .str.cat(may, sep=" || ", na_rep="-")
)

for i in launch_series.unique():
    print(f'"{i}":  , #{sum(launch_series == i)}')

"Illumina HiSeq || 5/22/2020 || covid || l":  , #136
"Illumina HiSeq || 6/23/2020 || сovid || l":  , #39
"Illumina HiSeq || 6/23/2020 || covid || l":  , #24
"Illumina HiSeq || 6/23/2020 || LE2 || l":  , #1
"Illumina HiSeq || 6/23/2020 || LE3 || l":  , #1
"Illumina HiSeq || 12/25/2020 || ces 88 || l":  , #108
"Illumina HiSeq || 12/25/2020 || ces 88 || ex_cov":  , #7
"MGI 2000 || 12/29/2020 || cov || l":  , #15
"MGI 2000 || 12/29/2020 || covid || ex_cov":  , #7
"Illumina HiSeq || 12/29/2020 || ces 89 || ex_cov":  , #45
"Illumina HiSeq || 12/29/2020 || ces 89 || l":  , #104
"MGI 2000 || 12/29/2020 || ces 89 || ex_cov":  , #1
"MGI 2000 || 12/29/2020 || cov || ex_cov":  , #8
"Illumina HiSeq || 12/29/2020 || covid || l":  , #1
"MGI 2000 || 12/29/2020 || covid || l":  , #1
"Illumina HiSeq || 12/29/2020 || covid || ex_cov":  , #1
"MGI 2000 || 1/25/2021 || ces-2 || ex_cov":  , #4
"MGI 2000 || 2/5/2021 || covid || ex_cov":  , #16
"MGI 2000 || 2/5/2021 || Б 26 || l":  , #4
"MGI 2000 || 3/10/2021 

## Unifying IDs 
(because there are discrepancies in the library names 
in the table for one launch)

In [9]:
launch_to_id = {
    "Illumina HiSeq || 5/22/2020 || covid || l": 63,  # 136
    "Illumina HiSeq || 6/23/2020 || сovid || l": 66,  # 39
    "Illumina HiSeq || 6/23/2020 || covid || l": 66,  # 24
    "Illumina HiSeq || 6/23/2020 || LE2 || l": np.nan,  # 1   2 nans, cause 2 ids equal to 6671
    "Illumina HiSeq || 6/23/2020 || LE3 || l": 66,  # 1
    "Illumina HiSeq || 12/25/2020 || ces 88 || l": 89,  # 108   _________???
    "Illumina HiSeq || 12/25/2020 || ces 88 || ex_cov": 88,  # 7 _________???
    "MGI 2000 || 12/29/2020 || cov || l": 91,  # 15
    "MGI 2000 || 12/29/2020 || covid || ex_cov": 91,  # 7
    "Illumina HiSeq || 12/29/2020 || ces 89 || ex_cov": 88,  # 45   _________???
    "Illumina HiSeq || 12/29/2020 || ces 89 || l": 88,  # 104       _________???
    "MGI 2000 || 12/29/2020 || ces 89 || ex_cov": 91,  # 1
    "MGI 2000 || 12/29/2020 || cov || ex_cov": 91,  # 8
    "Illumina HiSeq || 12/29/2020 || covid || l": 89,  # 1    _________???
    "MGI 2000 || 12/29/2020 || covid || l": 91,  # 1
    "Illumina HiSeq || 12/29/2020 || covid || ex_cov": 88,  # 1   _________???
    "MGI 2000 || 1/25/2021 || ces-2 || ex_cov": 112,  # 4
    "MGI 2000 || 2/5/2021 || covid || ex_cov": 112,  # 16
    "MGI 2000 || 2/5/2021 || Б 26 || l": 112,  # 4
    "MGI 2000 || 3/10/2021 || Б 28 || ex_cov": 112,  # 12
    "MGI 2000 || 3/10/2021 || Б 28 || l": 112,  # 12
    "MGI 2000 || 3/10/2021 || ces || ex_cov": 112,  # 3
    "MGI 2000 || 3/26/2021 || c || l": 112,  # 4
    "MGI 2000 || 3/26/2021 || c || ex_cov": 112,  # 16
    "MGI 2000 || 09.09.2020 BGI || covid || l": 68,  # 34
    "MGI 2000 || 09.09.2020 BGI || сovid || l": 68,  # 1
    "Illumina_HiSeq   MGI_2000 || Illumina???  09.09.2020_BGI || covid || l": 68,  # 1
    "Illumina_HiSeq   MGI_2000 || Illumina_22.05.2020 09.09.2020_BGI || covid || l": 68,  # 6
    "Illumina_HiSeq   MGI_2000 Illumina_HiSeq  || Illumina_22.05.2020 09.09.2020_BGI Illumina_25.12.2020 || covid || l": 88,  # 1
    "Illumina_HiSeq   MGI_2000  || Illumina_23.06.2020 09.09.2020_BGI || сovid || l": 68,  # 3
    "Illumina_HiSeq        MGI_2000 || Illumina_25.12.2020 09.09.2020_BGI || сovid || l": 88,  # 1
    "Illumina HiSeq || надо поднимать  || covid || l": np.nan,  # 1
    "Illumina HiSeq || - || covid || l": np.nan,  # 1
    "MGI 2000 || - || ces-2 || ex_cov": 114,  # 7
    "Illumina HiSeq || 4/28/2021 || covid || l": 104,  # 95
    "Illumina HiSeq || 4/28/2021 || сovid || l": 104,  # 1
    "Illumina HiSeq || 5/20/2021 || covid || l": 105,  # 3
    "Illumina HiSeq || 5/20/2021 || covid || ex_cov": 105,  # 128
}

In [10]:
columns_to_show = [
    "launch_id",
    "man_id",
    "blood_id",
    "unique_concat_id",
    "launch_blood_id",
]


def get_man_to_blood(data):
    seq = data["СЕКВЕНАТОР"]
    date = data["ДАТА ЗАПУСКА"]
    lib = data["НАЗВАНИЕ БИБЛИОТЕКИ (ЗАПУСКА)"]
    may = data["may_launch_special_id"]
    data["seq-r,date,lib"] = (
        seq.str.cat(date, sep=" || ", na_rep="-")
        .str.cat(lib, sep=" || ", na_rep="-")
        .str.cat(may, sep=" || ", na_rep="-")
    )

    checked_columns = [
        "seq-r,date,lib",
        "N ПАЦИЕНТА(КРОВЬ)",
        "КАРТА ПАЦ-ТА",
        "СЕКВЕНАТОР",
        "НАЗВАНИЕ БИБЛИОТЕКИ (ЗАПУСКА)",
        "blood_id",
    ]

    for feature in checked_columns:
        data = data[data[feature].notna()]

    needed_columns = [
        "seq-r,date,lib",
        "N ПАЦИЕНТА(КРОВЬ)",
        "КАРТА ПАЦ-ТА",
        "blood_id",
    ]
    id_and_launch_id = data
    id_and_launch_id["unique_concat_id"] = id_and_launch_id["seq-r,date,lib"]
    id_and_launch_id = id_and_launch_id.replace({"seq-r,date,lib": launch_to_id})
    id_and_launch_id = id_and_launch_id[id_and_launch_id["seq-r,date,lib"].notna()]

    id_and_launch_id["launch_blood_id"] = id_and_launch_id["seq-r,date,lib"].astype(
        int
    ).astype(str).replace(" ", "", regex=True) + id_and_launch_id["blood_id"].astype(
        str
    )
    id_and_launch_id = id_and_launch_id.drop(["N ПАЦИЕНТА(КРОВЬ)"], axis=1)
    id_and_launch_id["seq-r,date,lib"] = id_and_launch_id["seq-r,date,lib"].astype(int)
    id_and_launch_id["КАРТА ПАЦ-ТА"] = id_and_launch_id["КАРТА ПАЦ-ТА"].astype(int)
    id_and_launch_id = id_and_launch_id.rename(
        columns={
            "seq-r,date,lib": "launch_id",
            "КАРТА ПАЦ-ТА": "man_id",
            "launch_blood_id": "launch_blood_id",
        }
    )
    return id_and_launch_id


id_and_launch_id = get_man_to_blood(data)
print(id_and_launch_id.shape)
id_and_launch_id[columns_to_show].head()

(849, 144)


,launch_id,man_id,blood_id,unique_concat_id,launch_blood_id
0,63,82590,11,Illumina HiSeq || 5/22/2020 || covid || l,6311
1,63,89212,309,Illumina HiSeq || 5/22/2020 || covid || l,63309
2,63,104435,373,Illumina HiSeq || 5/22/2020 || covid || l,63373
3,63,107415,254,Illumina HiSeq || 5/22/2020 || covid || l,63254
4,63,108533,73,Illumina HiSeq || 5/22/2020 || covid || l,6373


#### some of the ids have errors:

In [11]:
# check:
for index, value in id_and_launch_id["launch_blood_id"].items():
    try:
        int(value)
    except ValueError:
        cur_data = pd.DataFrame([id_and_launch_id.loc[index]])
        display(cur_data[columns_to_show])

,launch_id,man_id,blood_id,unique_concat_id,launch_blood_id
415,88,644520,1200/1234,Illumina HiSeq || 12/29/2020 || ces 89 || l,881200/1234


,launch_id,man_id,blood_id,unique_concat_id,launch_blood_id
418,88,647463,1221/1244,Illumina HiSeq || 12/29/2020 || ces 89 || l,881221/1244


,launch_id,man_id,blood_id,unique_concat_id,launch_blood_id
476,88,631655,8()/67,Illumina HiSeq || 12/29/2020 || ces 89 || ex_cov,888()/67


In [12]:
## this is done by hand (checked by VCF IDs)
def solve_problems(id_and_launch_id):
    id_and_launch_id.at[415, "launch_blood_id"] = "891200"
    id_and_launch_id.at[415, "blood_id"] = "1200"
    id_and_launch_id.at[418, "launch_blood_id"] = "891221"
    id_and_launch_id.at[418, "blood_id"] = "1221"
    id_and_launch_id.at[476, "launch_blood_id"] = "888"
    id_and_launch_id.at[476, "blood_id"] = "8"
    return id_and_launch_id


id_and_launch_id = solve_problems(id_and_launch_id).set_index("man_id")
id_and_launch_id["man_id"] = id_and_launch_id.index
print(id_and_launch_id.shape)
id_and_launch_id[columns_to_show]

(849, 144)


,launch_id,man_id,blood_id,unique_concat_id,launch_blood_id
man_id,,,,,
82590,63,82590,11,Illumina HiSeq || 5/22/2020 || covid || l,6311
89212,63,89212,309,Illumina HiSeq || 5/22/2020 || covid || l,63309
104435,63,104435,373,Illumina HiSeq || 5/22/2020 || covid || l,63373
107415,63,107415,254,Illumina HiSeq || 5/22/2020 || covid || l,63254
108533,63,108533,73,Illumina HiSeq || 5/22/2020 || covid || l,6373
...,...,...,...,...,...
562954,105,562954,287,Illumina HiSeq || 5/20/2021 || covid || ex_cov,105287
695331,105,695331,288,Illumina HiSeq || 5/20/2021 || covid || ex_cov,105288
702652,105,702652,289,Illumina HiSeq || 5/20/2021 || covid || ex_cov,105289


## Old IDs mapping
Otherwise, we will have problems with the 88th and 89th launches (was obtained empirically).

In [13]:
mapping = pd.read_csv(OLD_MAPPING_FILE, index_col="man_id_map")
mapping["launch_blood_id_map"] = mapping["launch_blood_id_map"].astype(str)
mapping

,launch_blood_id_map,death_map
man_id_map,,
628465,88324,2
326997,893299,1
648682,91133,2
678479,91134,2
635694,91135,2
...,...,...
675169,893324,1
675506,893335,1
675767,893362,1


In [14]:
mapping_cols_to_show = [
    "launch_id",
    "blood_id",
    "unique_concat_id",
    "launch_blood_id",
    "launch_blood_id_map",
    "death_map",
]

In [15]:
new_data_ind = pd.concat([id_and_launch_id, mapping], axis=1, join="outer")
new_data_ind.launch_blood_id = new_data_ind.launch_blood_id.astype(str)
new_data_ind.launch_blood_id_map = new_data_ind.launch_blood_id_map.astype(str)
new_data_ind[mapping_cols_to_show]

,launch_id,blood_id,unique_concat_id,launch_blood_id,launch_blood_id_map,death_map
29478,114.0,120,MGI 2000 || - || ces-2 || ex_cov,114120,nan,NaN
68100,104.0,1862,Illumina HiSeq || 4/28/2021 || covid || l,1041862,nan,NaN
70747,105.0,237,Illumina HiSeq || 5/20/2021 || covid || ex_cov,105237,nan,NaN
74079,104.0,6476,Illumina HiSeq || 4/28/2021 || covid || l,1046476,nan,NaN
74599,105.0,216,Illumina HiSeq || 5/20/2021 || covid || ex_cov,105216,nan,NaN
...,...,...,...,...,...,...
703900,105.0,210,Illumina HiSeq || 5/20/2021 || covid || ex_cov,105210,nan,NaN
704479,105.0,158,Illumina HiSeq || 5/20/2021 || covid || ex_cov,105158,nan,NaN
704619,105.0,211,Illumina HiSeq || 5/20/2021 || covid || ex_cov,105211,nan,NaN
704809,105.0,157,Illumina HiSeq || 5/20/2021 || covid || ex_cov,105157,nan,NaN


In [16]:
new_data_ind[new_data_ind["launch_id"].isnull()][mapping_cols_to_show]

,launch_id,blood_id,unique_concat_id,launch_blood_id,launch_blood_id_map,death_map
457594,NaN,NaN,NaN,nan,63325,2.0


In [17]:
print(new_data_ind.shape)
new_data_ind = new_data_ind[~new_data_ind["launch_id"].isnull()]
print(new_data_ind.shape)

(850, 146)
(849, 146)


In [18]:
columns_to_show = ["launch_id", "man_id", "blood_id", "unique_concat_id", "final_id"]

In [19]:
new_data_ind["final_id"] = new_data_ind["launch_blood_id_map"]
new_data_ind.loc[new_data_ind.death_map.isnull(), "final_id"] = new_data_ind[
    new_data_ind.death_map.isnull()
]["launch_blood_id"]
new_data_ind["launch_id"] = new_data_ind["launch_id"].astype(int).astype(str)
new_data_ind["blood_id"] = new_data_ind["blood_id"].astype(int).astype(str)
new_data_ind["final_id"] = new_data_ind["final_id"].astype(int).astype(str)
new_data_ind = new_data_ind.drop(
    ["launch_blood_id", "launch_blood_id_map", "death_map"], axis=1
)
new_data_ind["man_id"] = new_data_ind.index
print(new_data_ind.shape)
new_data_ind[columns_to_show]

(849, 144)


,launch_id,man_id,blood_id,unique_concat_id,final_id
29478,114,29478,120,MGI 2000 || - || ces-2 || ex_cov,114120
68100,104,68100,1862,Illumina HiSeq || 4/28/2021 || covid || l,1041862
70747,105,70747,237,Illumina HiSeq || 5/20/2021 || covid || ex_cov,105237
74079,104,74079,6476,Illumina HiSeq || 4/28/2021 || covid || l,1046476
74599,105,74599,216,Illumina HiSeq || 5/20/2021 || covid || ex_cov,105216
...,...,...,...,...,...
703900,105,703900,210,Illumina HiSeq || 5/20/2021 || covid || ex_cov,105210
704479,105,704479,158,Illumina HiSeq || 5/20/2021 || covid || ex_cov,105158
704619,105,704619,211,Illumina HiSeq || 5/20/2021 || covid || ex_cov,105211
704809,105,704809,157,Illumina HiSeq || 5/20/2021 || covid || ex_cov,105157


Renamed samples:

In [20]:
new_data_ind[new_data_ind.launch_id + new_data_ind.blood_id != new_data_ind.final_id][
    columns_to_show
]

,launch_id,man_id,blood_id,unique_concat_id,final_id
92435,88,92435,3541,Illumina HiSeq || 12/29/2020 || ces 89 || l,893541
101400,89,101400,1821,Illumina HiSeq || 12/25/2020 || ces 88 || l,881821
119475,63,119475,2,Illumina HiSeq || 5/22/2020 || covid || l,892
128328,88,128328,3373,Illumina HiSeq || 12/29/2020 || ces 89 || l,893373
135790,88,135790,1062,Illumina HiSeq || 12/29/2020 || ces 89 || l,891062
...,...,...,...,...,...
675767,88,675767,3362,Illumina HiSeq || 12/29/2020 || ces 89 || l,893362
675777,88,675777,3359,Illumina HiSeq || 12/29/2020 || ces 89 || l,893359
675861,88,675861,3581,Illumina HiSeq || 12/29/2020 || ces 89 || l,893581
676788,91,676788,47,MGI 2000 || 12/29/2020 || ces 89 || ex_cov,8847


In [21]:
new_data_ind[new_data_ind.blood_id.apply(len) < 2][columns_to_show]

,launch_id,man_id,blood_id,unique_concat_id,final_id
117555,88,117555,9,Illumina HiSeq || 12/29/2020 || covid || ex_cov,889
119475,63,119475,2,Illumina HiSeq || 5/22/2020 || covid || l,892
537668,112,537668,9,MGI 2000 || 2/5/2021 || Б 26 || l,1129
568174,88,568174,4,Illumina HiSeq || 12/29/2020 || ces 89 || ex_cov,884
620762,63,620762,9,Illumina HiSeq || 5/22/2020 || covid || l,639
621011,88,621011,1,Illumina HiSeq || 12/29/2020 || ces 89 || ex_cov,881
625668,63,625668,3,Illumina HiSeq || 5/22/2020 || covid || l,893
626630,63,626630,7,Illumina HiSeq || 5/22/2020 || covid || l,637
627130,88,627130,2,Illumina HiSeq || 12/29/2020 || ces 89 || ex_cov,882
627146,88,627146,5,Illumina HiSeq || 12/29/2020 || ces 89 || ex_cov,885


In [22]:
new_data_ind.shape

(849, 144)

## Intersection

Let's intersect ids from phenotypes and ids from vcfs, to get final set.

In [23]:
launch_ids = list(new_data_ind["final_id"])
assert len(vcfs_ids) == len(set(vcfs_ids))
assert len(launch_ids) == len(set(launch_ids))

launch_ids = set(launch_ids)
vcfs_ids = set(vcfs_ids)


def return_starts_with(array, substr):
    new_ar = []
    for i in array:
        ii = str(i)
        true_start = 3 if ii[0] == 1 else 2
        if ii.startswith(substr):
            new_ar.append(ii[true_start:])
    return new_ar


def return_array_print_len(array):
    print(f"{len(array)}")
    return array


intersection = launch_ids & vcfs_ids

for ar, name in zip(
    [launch_ids, vcfs_ids, intersection], ["pheno", "vcf", "intersection"]
):
    print(name)
    for i in ["63", "66", "68", "88", "89", "91", "104", "105", "112", "114"]:
        print(f"Launch {i}: ", end="")
        array = return_array_print_len(return_starts_with(ar, i))
    #         print(np.sort(array))
    print()
print(f"Intersected samples: {len(intersection)}/{len(vcfs_ids)}")

intersection_list = [i for i in intersection]

pheno
Launch 63: 134
Launch 66: 64
Launch 68: 46
Launch 88: 132
Launch 89: 139
Launch 91: 31
Launch 104: 95
Launch 105: 131
Launch 112: 70
Launch 114: 7

vcf
Launch 63: 144
Launch 66: 69
Launch 68: 47
Launch 88: 156
Launch 89: 140
Launch 91: 29
Launch 104: 96
Launch 105: 134
Launch 112: 71
Launch 114: 7

intersection
Launch 63: 134
Launch 66: 63
Launch 68: 46
Launch 88: 132
Launch 89: 138
Launch 91: 24
Launch 104: 95
Launch 105: 131
Launch 112: 70
Launch 114: 7

Intersected samples: 840/893


In [24]:
columns_to_show = ["launch_id", "man_id", "blood_id", "unique_concat_id"]

In [25]:
final_data = new_data_ind[
    new_data_ind["final_id"].apply(lambda x: x in intersection)
].set_index("final_id")
final_data[columns_to_show]

,launch_id,man_id,blood_id,unique_concat_id
final_id,,,,
114120,114,29478,120,MGI 2000 || - || ces-2 || ex_cov
1041862,104,68100,1862,Illumina HiSeq || 4/28/2021 || covid || l
105237,105,70747,237,Illumina HiSeq || 5/20/2021 || covid || ex_cov
1046476,104,74079,6476,Illumina HiSeq || 4/28/2021 || covid || l
105216,105,74599,216,Illumina HiSeq || 5/20/2021 || covid || ex_cov
...,...,...,...,...
105210,105,703900,210,Illumina HiSeq || 5/20/2021 || covid || ex_cov
105158,105,704479,158,Illumina HiSeq || 5/20/2021 || covid || ex_cov
105211,105,704619,211,Illumina HiSeq || 5/20/2021 || covid || ex_cov


In [26]:
files_data = pd.DataFrame(
    tuple_to_rename, columns=["old_file_name", "new_file_name", "final_id"]
).set_index("final_id")
files_data = files_data.drop(["new_file_name"], axis=1)
files_data.old_file_name = files_data.old_file_name.apply(
    lambda x: x.replace(".g.vcf.gz", "")
    .replace(".out", "")
    .replace(".g.vcf.gz", "")
    .replace(".recal", "")
    .replace(".dedup", "")
)
files_data

,old_file_name
final_id,
104555,ces_104.sample_L1-1-555
1046472,ces_104.sample_L1-10-6472
1046474,ces_104.sample_L1-11-6474
1046475,ces_104.sample_L1-12-6475
1046476,ces_104.sample_L1-13-6476
...,...
114118,ces_114.sample_Ex_cov_118.lane_4
114119,ces_114.sample_Ex_cov_119.lane_4
114120,ces_114.sample_Ex_cov_120.lane_4


Saving files for vcfs transformation:

In [27]:
for_transformation = pd.concat([files_data, final_data], axis=1, join="inner")[
    ["old_file_name", "man_id"]
]
for_transformation.to_csv(FOR_TRANSFORMATION_TO_SAVE)
for_transformation

,old_file_name,man_id
final_id,,
104555,ces_104.sample_L1-1-555,597619
1046472,ces_104.sample_L1-10-6472,263436
1046474,ces_104.sample_L1-11-6474,695634
1046475,ces_104.sample_L1-12-6475,694025
1046476,ces_104.sample_L1-13-6476,74079
...,...,...
114118,ces_114.sample_Ex_cov_118.lane_4,660644
114119,ces_114.sample_Ex_cov_119.lane_4,660141
114120,ces_114.sample_Ex_cov_120.lane_4,29478


Saving file for future GWAS:

In [28]:
for_gwas = pd.concat([files_data, final_data], axis=1, join="inner").drop(
    ["old_file_name"], axis=1
)
for_gwas.to_csv(FOR_GWAS_TO_SAVE)
for_gwas

,"ШТОРМ 1-да, 0-нет",от куда пациент,"Пол ( 1-м, 2-ж)",Количество суток с начала заболевания до госпитализации,Дата начала лечения,День заболевания,"Исходы (1 умер, 2 выписка)","Степень тяжести заболевания (1-легкая, 2-средняя, 3-тяжелая и кр.тяжелая)",ДАТА РОЖДЕНИЯ (ВОЗРАСТ),СЕКВЕНАТОР,...,Фиброз межальвеолярных перегородок,Интерстициальный отёк,Утолщение межальвеолярных перегородок за счет склероза,Лимфо-макрофагальная инфильтрация межальвеолярных перегородок,Аспирационная пневмония (инородные частицы в просвете терминальной бронхиолы),blood_id,may_launch_special_id,launch_id,unique_concat_id,man_id
final_id,,,,,,,,,,,,,,,,,,,,,
104555,0,NaN,2,NaN,NaN,NaN,NaN,NaN,03.04.1998,Illumina HiSeq,...,NaN,NaN,NaN,NaN,NaN,555,l,104,Illumina HiSeq || 4/28/2021 || covid || l,597619
1046472,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,06.08.1969,Illumina HiSeq,...,NaN,NaN,NaN,NaN,NaN,6472,l,104,Illumina HiSeq || 4/28/2021 || covid || l,263436
1046474,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,07.02.1955,Illumina HiSeq,...,NaN,NaN,NaN,NaN,NaN,6474,l,104,Illumina HiSeq || 4/28/2021 || covid || l,695634
1046475,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.10.1971,Illumina HiSeq,...,NaN,NaN,NaN,NaN,NaN,6475,l,104,Illumina HiSeq || 4/28/2021 || covid || l,694025
1046476,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,05.09.1959,Illumina HiSeq,...,NaN,NaN,NaN,NaN,NaN,6476,l,104,Illumina HiSeq || 4/28/2021 || covid || l,74079
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114118,1,3 корпус,2,8.0,8/4/2020,12,2,2,57,MGI 2000,...,NaN,NaN,NaN,NaN,NaN,118,ex_cov,114,MGI 2000 || - || ces-2 || ex_cov,660644
114119,1,3 корпус,2,7.0,7/29/2020,8,2,2,52,MGI 2000,...,NaN,NaN,NaN,NaN,NaN,119,ex_cov,114,MGI 2000 || - || ces-2 || ex_cov,660141
114120,1,3 корпус,2,3.0,8/4/2020,7,2,2,65,MGI 2000,...,NaN,NaN,NaN,NaN,NaN,120,ex_cov,114,MGI 2000 || - || ces-2 || ex_cov,29478


Transformt columns to int's/nans

In [29]:
def trans(x):
    try:
        return str(int(x))
    except:
        return np.nan

Don't know about them:

In [30]:
dont_know = set(data["КАРТА ПАЦ-ТА"].apply(trans)) - set(
    for_transformation.man_id.astype(str)
)
dont_know

{'231824',
 '331661',
 '556834',
 '623408',
 '651305',
 '651337',
 '659688',
 '664955',
 '665374',
 '667078',
 '667437',
 '667591',
 nan}

In [31]:
with open(MAPPING_DONT_KNOW_ABOUT_THEM, "w") as dk_file:
    dk_file.write(
        str(dont_know)
        .replace("{", "")
        .replace("}", "")
        .replace("'", "")
        .replace(", ", "\n")
    )